# Abordagem exaustiva

## Geração do grafo

In [ ]:
%%writefile gera_grafo1.py
import networkx as nx
import random

# Parâmetros
num_vertices = 100  # Número de vértices no grafo
probabilidade_conexao = 0.7  # Probabilidade de haver uma aresta entre dois vértices (ajuste conforme necessário)

# Crie um grafo aleatório densamente conectado
grafo = nx.fast_gnp_random_graph(num_vertices, probabilidade_conexao)

# Nome do arquivo de saída
nome_arquivo = "grafo1.txt"

# Abra o arquivo para escrita
with open(nome_arquivo, 'w') as arquivo:
    # Escreva a quantidade de vértices e número de arestas na primeira linha
    arquivo.write(f"{num_vertices} {grafo.number_of_edges()}\n")

    # Escreva as arestas no formato de lista de adjacência
    for aresta in grafo.edges():
        arquivo.write(f"{aresta[0]+1} {aresta[1]+1}\n")  # +1 para ajustar os índices (começando em 1)

print(f"Grafo densamente conectado gerado e salvo em '{nome_arquivo}'.")

# Image of the generated graph
import matplotlib.pyplot as plt

# Alterar edges do grafo para edge + 1
novas_arestas = [(u + 1, v + 1) for u, v in grafo.edges()]
novo_grafo = nx.Graph()
novo_grafo.add_edges_from(novas_arestas)
novo_grafo.add_nodes_from([n + 1 for n in grafo.nodes()])

# Desenhar o grafo
nx.draw(novo_grafo, with_labels=True)
plt.show()

## Código implementado

In [ ]:
%%writefile abordagem_exaustiva.cpp
#include <iostream>
#include <vector>
#include <fstream>
#include <algorithm>
#include <functional>

using namespace std;

vector<vector<int> > LerGrafo(const string& nomeArquivo, int& numVertices) {
    ifstream arquivo(nomeArquivo);
    int numArestas;
    arquivo >> numVertices >> numArestas;
    vector<vector<int> > grafo(numVertices, vector<int>(numVertices, 0));
    for (int i = 0; i < numArestas; ++i) {
        int u, v;
        arquivo >> u >> v;
        grafo[u - 1][v - 1] = 1;
        grafo[v - 1][u - 1] = 1;
    }
    arquivo.close();
    return grafo;
}

bool VerificaClique(const vector<vector<int> >& grafo, const vector<int>& clique) {
    for (int i = 0; i < clique.size(); i++) {
        for (int j = i + 1; j < clique.size(); j++) {
            if (grafo[clique[i]][clique[j]] == 0) { return false; }
        }
    }
    return true;
}

void EncontrarCliqueMaxima(const vector<vector<int> >& grafo, int numVertices, vector<int>& cliqueMaxima) {
    vector<int> atualClique;

    // Função recursiva para construir cliques
    auto backtrack = [&](int v, auto& backtrack_ref) -> void {
        // Atualiza se encontrar um clique maior
        if (atualClique.size() > cliqueMaxima.size()) { cliqueMaxima = atualClique; }

        for (int i = v; i < numVertices; i++) {
            atualClique.push_back(i);
            if (VerificaClique(grafo, atualClique)) { backtrack_ref(i + 1, backtrack_ref); }
            atualClique.pop_back();
        }
    };

    backtrack(0, backtrack);  // Inicia com o primeiro vértice
}

int main() {
    int numVertices = 100;
    vector<vector<int> > grafo = LerGrafo("grafo_teste.txt", numVertices);
    vector<int> cliqueMaxima;
    EncontrarCliqueMaxima(grafo, numVertices, cliqueMaxima);
    cout << "Clique máxima arquivo saída: ";
    for(int i = 0; i < cliqueMaxima.size(); i++){ cout << cliqueMaxima[i] + 1 << " "; }
    cout << endl;
    return 0;
}

## Verificação da clique

In [ ]:
%%writefile verificacao_da_clique.py
import networkx as nx

# Abrir o arquivo e pular a primeira linha
with open("grafo_teste.txt", 'r') as arquivo:
    next(arquivo)  # Pula a primeira linha

    # Lê o grafo a partir das linhas restantes
    G = nx.parse_adjlist(arquivo)

# Encontrar todas as cliques maximais
cliques_maximais = list(nx.find_cliques(G))

# Encontrar a clique máxima (a maior)
clique_maxima = max(cliques_maximais, key=len)

# Salvar as cliques maximais em um arquivo
with open("clique_teste.txt", 'w') as arquivo:
    arquivo.write(f"Cliques maximais encontradas\n")
    for clique in cliques_maximais:
        arquivo.write(f"{' '.join(map(str, clique))}\n")
    arquivo.write(f"\nClique máxima encontrada\n")
    arquivo.write(f"{len(clique_maxima)} = {' '.join(map(str, clique_maxima))}\n")

## Análise e Pseudo-código

Aproveitando para pensar se é possível adotar alguma heurística, ao ordenar os nós em função do grau de adjacência ... 


Com um tempo hábil de XXX minutos, foi possível rodar o problema com um tamanho YYY.


Para otimizar essa tarefa exaustiva, pode-se observar o pseudo-código de uma heurística a seguir:

# Esse é o pseudo-código do enunciado, modificá-lo

```
Função EncontrarCliqueMaxima(grafo, numVertices)
    cliqueMaxima = ListaVazia()
    candidatos = ListaDeNós()  # Inicialmente, todos os nós são candidatos

    Para cada i de 0 até numVertices - 1 Faça
        Adicione i à lista de candidatos

    Enquanto candidatos não estiver vazia Faça
        v = Último elemento de candidatos
        Remova o último elemento de candidatos

        podeAdicionar = Verdadeiro

        Para cada u em cliqueMaxima Faça
            Se grafo[u][v] == 0 Então
                podeAdicionar = Falso
                Pare o loop
            Fim Se
        Fim Para

        Se podeAdicionar for Verdadeiro Então
            Adicione v a cliqueMaxima
            novosCandidatos = ListaDeNós()

            Para cada u em candidatos Faça
                adjacenteATodos = Verdadeiro

                Para cada c em cliqueMaxima Faça
                    Se grafo[u][c] == 0 Então
                        adjacenteATodos = Falso
                        Pare o loop
                    Fim Se
                Fim Para

                Se adjacenteATodos for Verdadeiro Então
                    Adicione u a novosCandidatos
                Fim Se
            Fim Para

            candidatos = novosCandidatos
        Fim Se
    Fim Enquanto

    Retorne cliqueMaxima
Fim Função


```

É possível implementar alguma poda? Algum critério que evite calcular um nó, dado que você já descobriu uma clique maior?
